In [87]:
!nvidia-smi


Tue Mar 28 22:44:02 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.41                 Driver Version: 531.41       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti    WDDM | 00000000:01:00.0 Off |                  N/A |
| N/A   50C    P8                5W /  N/A|    386MiB /  4096MiB |      2%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [88]:
import tensorflow as tf
print(tf.__version__)

2.11.0


#Importing Libraries

In [107]:
from tensorflow.keras.layers import Input,Lambda,Dense,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
import cv2
import numpy as np
from glob import glob


In [90]:
ROOT_PATH=r'C:\Users\rctuh\Desktop\minor projects\Vgg16_Image_Classifier\cats_and_dogs\cats_and_dogs'

In [91]:
pwd

'C:\\Users\\rctuh\\Desktop\\minor projects\\Vgg16_Image_Classifier\\cats_and_dogs\\cats_and_dogs'

In [108]:
import os 
os.chdir(ROOT_PATH)
os.getcwd()

'C:\\Users\\rctuh\\Desktop\\minor projects\\Vgg16_Image_Classifier\\cats_and_dogs\\cats_and_dogs'

In [109]:
train_path=r"C:\Users\rctuh\Desktop\minor projects\Vgg16_Image_Classifier\cats_and_dogs\cats_and_dogs\train"
test_path=r"C:\Users\rctuh\Desktop\minor projects\Vgg16_Image_Classifier\cats_and_dogs\cats_and_dogs\test"
val_path=r"C:\Users\rctuh\Desktop\minor projects\Vgg16_Image_Classifier\cats_and_dogs\cats_and_dogs\validation"

In [110]:
x_train=[]

for folder in os.listdir(train_path):

    sub_path=train_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(224,224))

        x_train.append(img_arr)

In [111]:
x_test=[]

for folder in os.listdir(test_path):

    sub_path=test_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(224,224))

        x_test.append(img_arr)

In [112]:
x_val=[]

for folder in os.listdir(val_path):

    sub_path=val_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(224,224))

        x_val.append(img_arr)

In [113]:
train_x=np.array(x_train)
test_x=np.array(x_test)
val_x=np.array(x_val)
train_x=train_x/255.0
test_x=test_x/255.0
val_x=val_x/255.0

#Resizing all images

In [114]:
IMAGE_SIZE=[224,224]

#Importing VGG16 library and add preprocesing layer in front of the VGG16.
#Here we will se ImageNet weights

In [115]:
vgg16=VGG16(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)

include_top= False means that we will cut off the last dense layer of pre-trained model of VGG16 as it was trained for 1000 output categories of imagenet whereas we only have 3 categories. We also cut off the 1st layer because the input image and the size can be of my choice.

In [116]:
for layers in vgg16.layers:
  print(layers)

###Setting layer.trainable to False moves all the layer's weights from trainable to non-trainable. This is called "freezing" the layer: the state of a frozen layer won't be updated during training (either when training with fit() or when training with any custom loop that relies on trainable_weights to apply gradient updates).

In [117]:
for layer in vgg16.layers:
  layer.trainable=False

In [118]:
for layer in vgg16.layers:
  print(layer.name,layer.trainable)

input_5 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 False
block5_conv2 False
block5_conv3 False
block5_pool False


#Vgg16 Model Summary

In [119]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

#Building the Final Model 


In [120]:
model = Sequential()

model.add(vgg16)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(2,activation='Softmax'))


#View the structure of final model

In [121]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_4 (Flatten)         (None, 25088)             0         
                                                                 
 dense_8 (Dense)             (None, 256)               6422784   
                                                                 
 dense_9 (Dense)             (None, 2)                 514       
                                                                 
Total params: 21,137,986
Trainable params: 6,423,298
Non-trainable params: 14,714,688
_________________________________________________________________


#Specifying the Cost and Optimization Methods

In [122]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics='accuracy'
)

#Normalization of values to 255

#Using the Image Data Generator to import images from dataset

In [123]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen= ImageDataGenerator(rescale=1./255,
                                  shear_range =0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen =  ImageDataGenerator(rescale = 1./255)

In [124]:
pwd

'C:\\Users\\rctuh\\Desktop\\minor projects\\Vgg16_Image_Classifier\\cats_and_dogs\\cats_and_dogs'

In [63]:
training_set=train_datagen.flow_from_directory(r'C:\Users\rctuh\Desktop\minor projects\Vgg16_Image_Classifier\cats_and_dogs\cats_and_dogs\train',
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode='categorical')


Found 2000 images belonging to 2 classes.


In [125]:
test_set=test_datagen.flow_from_directory(r'C:\Users\rctuh\Desktop\minor projects\Vgg16_Image_Classifier\cats_and_dogs\cats_and_dogs\test',
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode='categorical'
                                            )

Found 1000 images belonging to 2 classes.


In [126]:
val_set=val_datagen.flow_from_directory(r'C:\Users\rctuh\Desktop\minor projects\Vgg16_Image_Classifier\cats_and_dogs\cats_and_dogs\validation',
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode='categorical'
                                            )

Found 50 images belonging to 2 classes.


In [127]:
train_y=training_set.classes
test_y=test_set.classes
val_y=val_set.classes

In [128]:
training_set.class_indices
train_y.shape,test_y.shape,val_y.shape

((2000,), (1000,), (50,))

#Early stopping to avoid overfitting of model

In [129]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)


#Model Fitting

In [130]:
r= model.fit(
    training_set,
    validation_data=val_set,
    epochs=10,
    callbacks=[early_stop],
    steps_per_epoch=len(training_set),
    validation_steps=len(val_set)
)

Epoch 1/10
63/63 [==============================] - 148s 2s/step - loss: 0.9189 - accuracy: 0.7885 - val_loss: 0.1605 - val_accuracy: 0.9400
Epoch 2/10
63/63 [==============================] - 151s 2s/step - loss: 0.2227 - accuracy: 0.9050 - val_loss: 0.1992 - val_accuracy: 0.9400
Epoch 3/10
63/63 [==============================] - 145s 2s/step - loss: 0.1768 - accuracy: 0.9335 - val_loss: 0.1470 - val_accuracy: 0.9600
Epoch 4/10
63/63 [==============================] - 153s 2s/step - loss: 0.1413 - accuracy: 0.9395 - val_loss: 0.2626 - val_accuracy: 0.9200
Epoch 5/10
63/63 [==============================] - 148s 2s/step - loss: 0.1549 - accuracy: 0.9400 - val_loss: 0.4678 - val_accuracy: 0.9000
Epoch 6/10
63/63 [==============================] - 144s 2s/step - loss: 0.1048 - accuracy: 0.9570 - val_loss: 0.3226 - val_accuracy: 0.9200
Epoch 7/10
63/63 [==============================] - 145s 2s/step - loss: 0.0957 - accuracy: 0.9640 - val_loss: 0.2770 - val_accuracy: 0.9000
Epoch 8/10
55

#Plotting the Losses

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(r.history['loss'], label='Train Loss')
plt.plot(r.history['val_loss'], label='Val Loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

#Plotting Accuracy 


In [ ]:
plt.plot(r.history['accuracy'], label='Train Acc')
plt.plot(r.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

#Saving the File as .h5 Type


In [ ]:
from tensorflow.keras.models import load_model 
model.save('model_vgg16.h5')

#Predictions

In [ ]:
y_pred=model.predict(test_set)


In [ ]:
y_pred

In [ ]:
import numpy as np
y_pred=np.argmax(y_pred,axis=1)

In [ ]:
y_pred

In [ ]:
from tensorflow.keras.preprocessing import image


In [ ]:
model=load_model('model_vgg16.h5')
img_path=r'C:\Users\rctuh\Desktop\minor projects\Vgg16_Image_Classifier\cats_and_dogs\cats_and_dogs\validation\dogs\dog.2012.jpg'
img=image.load_img(img_path,target_size=(224,224))


In [ ]:
x=image.img_to_array(img)

In [ ]:
Z=plt.imread(img_path)

In [ ]:
plt.imshow(Z)

In [ ]:
x.shape


In [ ]:
x=x/255

In [ ]:
from keras.applications.vgg16 import preprocess_input 

In [ ]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

In [ ]:
model.predict(img_data)

In [ ]:
result=np.argmax(model.predict(img_data),axis=1)

In [ ]:
result

In [ ]:
if result[0] == 1:
  prediction = "The Given Image is of a Dog"
  print(prediction)
else:
  prediction="The Given Image is of a cat"
  print(prediction)
  

Losses and Accuracy 


In [ ]:
 from sklearn.metrics import accuracy_score
 from sklearn.metrics import f1_score
 from sklearn.metrics import roc_auc_score


In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
f1_score(test_y,y_pred)

In [ ]:
accuracy_score(test_y, y_pred)

In [ ]:
roc_auc_score(test_y,y_pred)

In [ ]:
print(classification_report(y_pred,test_y))

In [ ]:
print(confusion_matrix(y_pred,test_y))